# Tarea 6
# Métodos numéricos y optimización
## Juan Pablo Herrera - 108353

### Instrucciones

Dirígite a la rama de master a la ruta analisis-numerico-computo-cientifico/C/BLAS/ejemplos/.

Investiga* sobre la subrutina de Fortran dgemv (parámetros que recibe y la salida).

*Para sus investigaciones utilicen las referencias de los README's:

analisis-numerico-computo-cientifico/C/BLAS/

analisis-numerico-computo-cientifico/C/LAPACK/

La tarea se hace de forma individual y personal.
    En la carpeta analisis-numerico-computo-cientifico/C/BLAS/ejemplos/level2/ ejecuta el programa dgemv_mult_mat_vec.c y realiza pruebas con diferentes matrices y vectores definidos por ti.

In [1]:
%%bash
sudo apt-get update
sudo apt-get -y install libblas-dev liblapack-dev

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [831 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [11.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [48.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1360 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [34.6 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [855 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [7904 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1149 kB]
Get:13

debconf: delaying package configuration, since apt-utils is not installed


### Pregunta 1
**Investiga* sobre la subrutina de Fortran dgemv (parámetros que recibe y la salida).**

La función **dgemv** de Fortran resuelve una la operación de multiplicar un vector $x$ por una matriz $A$ y sumar otro vector $y$. A ambos operandos se les puede multiplicar por una constante nombradas como $\alpha$ o $\beta$. Resuelve uno de los dos siguientes casos:  
$$\begin{equation}
    y=\alpha A x + \beta y
    \tag{1}
    \end{equation}$$ o  
$$\begin{equation}
    y=\alpha A^T X + \beta y
    \tag{2}
    \end{equation}$$  

La función recibe los siguientes argumentos:
* TRANS (char)- Puede tomar 3 valores: 'N' para indicar que la operación a realizar es la ecuación $1$. Los valores 'T' o 'C' indican que se realizará la ecuación $2$. Las entradas pueden ser mayúsculas o minúsculas indistinamente. 
* M (int)- Especifíca el número de filas que tiene la matriz $A$. Debe de ser mayor o igual que cero.  
* N(int)- Especifíca el número de columnas que tiene la matriz $A$. Debe de ser mayor o igual que cero.   
* ALPHA (double)- Indica el escalar que multiplica la matriz $A$.
* A (double)- Es un array de dimensión $(LDA,N)$. La parte $m$ por $n$ debe de contener los coeficientes de la matriz.  
* LDA (int)- Especifíca la dimensión de $A$ como se declara en el programa que llama la función. Debe de ser com mínimo $max(1,m)$.
* X (double)- El vector a operar. Debe de tener como mínimo las dimensiones:   
$$(1+(n-1)*|INCX|) \text{ when } TRANS = 'N'$$ o
$$(1+(m-1)*|INCX|) \text{ when } TRANS = 'C' \text{ or } TRANS = 'T' $$   

* INCX (int)- Incremento entre cada elemento consecutivo del arreglo $X$.
* Beta (double)- Indica el escalar que multiplica al vector $Y$.
* Y (double)- El vector a operar. Debe de tener como mínimo las dimensiones:   
$$(1+(m-1)*|INCX|) \text{ when } TRANS = 'N'$$ o
$$(1+(n-1)*|INCX|) \text{ when } TRANS = 'C' \text{ or } TRANS = 'T' $$  
* INCY(int)- Incremento entre cada elemento consecutivo del arreglo $Y$.  

La función devuelve un vector de la misma dimensión que el arreglo $Y$

### Pregunta 2
**En la carpeta analisis-numerico-computo-cientifico/C/BLAS/ejemplos/level2/ ejecuta el programa dgemv_mult_mat_vec.c y realiza pruebas con diferentes matrices y vectores definidos por ti.**

In [3]:
%%bash 
echo 1 0 > v.txt
echo 0 1.5 >A.txt
echo 4 -5 >> A.txt
echo -1 2.5 >> A.txt

In [42]:
%%file dgemv_mult_mat_vec.c
#include<stdio.h>
#include<stdlib.h>
#include"definiciones.h"
#define A_matriz "A.txt" //de tamaño MxN
#define v_vector "v.txt" //de tamaño Nx1
extern void dgemv_(char *transpose_a, int *m, int *n, double *alpha, double *a, int *lda, double *x, int *incx, double *beta, double *y, int *incy);
int main(int argc, char *argv[]){
	arreglo_2d_T A;
	arreglo_1d_T v, v_resultado;
	int M=atoi(argv[1]);
	int N=atoi(argv[2]);
	int incx=1;
	double ALPHA, BETA;
    ALPHA = 1.0;
   	BETA = 0.0;
	A=malloc(sizeof(*A));
	v=malloc(sizeof(*v));
	v_resultado=malloc(sizeof(*v_resultado));

	renglones(A)=M;
	columnas(A)=N;
	renglones_vector(v)=N;
	renglones_vector(v_resultado) = M;
	entradas(A)=malloc(renglones(A)*columnas(A)*sizeof(double));
	inicializa_matriz(A,A_matriz);

	entradas_vector(v)=malloc(N*sizeof(double));
	inicializa_vector(v,v_vector);

	entradas_vector(v_resultado)=calloc(M,sizeof(double));
	printf("matriz 1:\n");
	imprime_matriz(A);
	printf("------------\n");
	printf("vector :\n");
	imprime_vector(v);
	printf("------------\n");
	printf("vector resultado:\n");

	dgemv_("No transpose", &M, &N, &ALPHA, entradas(A), &M, entradas_vector(v), &incx, &BETA, entradas_vector(v_resultado),&incx);
	imprime_vector(v_resultado);
	free(entradas(A));
	free(A);
	free(entradas_vector(v));
	free(v);
	free(entradas_vector(v_resultado));
	free(v_resultado);
	return 0;
}

Overwriting dgemv_mult_mat_vec.c


In [39]:
%%bash
gcc -Wall dgemv_mult_mat_vec.c funciones.c -o programa.out -lblas

In [40]:
%%bash
./programa.out 3 2

matriz 1:
matriz[0][0]=0.00000	matriz[0][1]=1.50000
matriz[1][0]=4.00000	matriz[1][1]=-5.00000
matriz[2][0]=-1.00000	matriz[2][1]=2.50000
------------
vector :
vector[0]=1.00000
vector[1]=0.00000
------------
vector resultado:
vector[0]=0.00000
vector[1]=4.00000
vector[2]=-1.00000


In [25]:
%%bash 
echo 1 0 3 7 >> v.txt
echo 0 1.5 >A.txt
echo 4 -5 >> A.txt
echo -1 2.5 >> A.txt

In [26]:
%%bash
./programa.out 3 2

matriz 1:
matriz[0][0]=0.00000	matriz[0][1]=1.50000
matriz[1][0]=4.00000	matriz[1][1]=-5.00000
matriz[2][0]=-1.00000	matriz[2][1]=2.50000
------------
vector :
vector[0]=1.00000
vector[1]=0.00000
------------
vector resultado:
vector[0]=0.00000
vector[1]=4.00000
vector[2]=-1.00000


Si le cambiamos las dimensiones en la entrada solamente toma en cuenta los primeros números que le alimentamos.

In [28]:
%%bash
./programa.out 5 3

matriz 1:
matriz[0][0]=0.00000	matriz[0][1]=1.50000	matriz[0][2]=4.00000
matriz[1][0]=-5.00000	matriz[1][1]=-1.00000	matriz[1][2]=2.50000
matriz[2][0]=0.00000	matriz[2][1]=0.00000	matriz[2][2]=0.00000
matriz[3][0]=0.00000	matriz[3][1]=0.00000	matriz[3][2]=0.00000
matriz[4][0]=0.00000	matriz[4][1]=0.00000	matriz[4][2]=0.00000
------------
vector :
vector[0]=1.00000
vector[1]=0.00000
vector[2]=3.00000
------------
vector resultado:
vector[0]=12.00000
vector[1]=2.50000
vector[2]=0.00000
vector[3]=0.00000
vector[4]=0.00000


Si le ponemos dimensiones mayores a los número que tenemos en los archivos rellena los espacios con ceros.  

El vector de salida tiene la dimensión de las columnas de la matriz.  

Hay que notar que en este ejemplo usamos el parámetro $\beta=0$

### Pregunta 3
**Haz un programa que utilice la subrutina dger de Fortran.**

La función **dger** relaiza la operación:  
$$A:=\alpha x y^T + A$$  
donde $\alpha$ es un escalar, $x$ es un vector de $m$ elementos, $y$ es un vector de $n$ elementos y $A$ es una matriz de $mxn$.  
La función recibe los parámetros:  
* M (int)- Filas de la matriz A  
* N (int)- Columnas de la matriz A  
* ALPHA (double)- Escalar que multiplica al vector $x$
* X (double)- Arreglo de dimensión m como mínimo
* INCX (int)- Distancia entre cada elemento de $x$
* Y (double)- Arreglo de dimensión $n$
* INCY (int)- Distancia entre cada elemento de $y$
* LDA (int)- Especifíca la primera dimensión de la matriz de salida
* A (double)- Matriz de coeficientes para sumar a la operación.

La salida es la matriz $A$ de dimensiones $(LDA,N)$



In [53]:
%%file dger.c
#include<stdio.h>
#include<stdlib.h>
#include"definiciones.h"
#define y_vector "y.txt" //de tamaño Nx1
#define x_vector "x.txt" //de tamaño Mx1

extern void dger_(int *m, int *n, double *alpha, double *x, int *incx, double *y, int *incy, double *a, int *lda);
int main(int argc, char *argv[]){
	arreglo_2d_T A;
	arreglo_1d_T x, y;
	int M=atoi(argv[1]);
	int N=atoi(argv[2]);
	int incx=1;
	double ALPHA;
    ALPHA = 1.0;
    
	A=malloc(sizeof(*A));
	x=malloc(sizeof(*x));
	y=malloc(sizeof(*y));

	renglones(A)=M;
	columnas(A)=N;
	renglones_vector(x)=M;
	renglones_vector(y) = N;
	entradas_vector(y)=malloc(N*sizeof(double));
	inicializa_vector(y,y_vector);

	entradas_vector(x)=malloc(M*sizeof(double));
	inicializa_vector(x,x_vector);

	
    entradas(A)=calloc(M*N,sizeof(double));
    printf("vector x:\n");
	imprime_vector(x);
	printf("------------\n");
	printf("vector y:\n");
	imprime_vector(y);
	printf("------------\n");
	printf("vector resultado:\n");

	dger_(&M, &N, &ALPHA, entradas_vector(x), &incx, entradas_vector(y),&incx, entradas(A), &M);
	imprime_matriz(A);
	
    free(entradas(A));
	free(A);
	free(entradas_vector(x));
	free(x);
	free(entradas_vector(y));
	free(y);
	return 0;
}

Overwriting dger.c


In [54]:
%%bash
gcc -Wall dger.c funciones.c -o prog_dger.out -lblas

In [55]:
%%bash 
echo 1 2 3 > x.txt
echo 4 5 >y.txt

In [56]:
%%bash
./prog_dger.out 3 2

vector x:
vector[0]=1.00000
vector[1]=2.00000
vector[2]=3.00000
------------
vector y:
vector[0]=4.00000
vector[1]=5.00000
------------
vector resultado:
matriz[0][0]=4.00000	matriz[0][1]=5.00000
matriz[1][0]=8.00000	matriz[1][1]=10.00000
matriz[2][0]=12.00000	matriz[2][1]=15.00000


In [57]:
%%bash 
echo 1 2 -3 > x.txt
echo 0 5 -7 0 1 >y.txt

In [58]:
%%bash
./prog_dger.out 3 5

vector x:
vector[0]=1.00000
vector[1]=2.00000
vector[2]=-3.00000
------------
vector y:
vector[0]=0.00000
vector[1]=5.00000
vector[2]=-7.00000
vector[3]=0.00000
vector[4]=1.00000
------------
vector resultado:
matriz[0][0]=0.00000	matriz[0][1]=5.00000	matriz[0][2]=-7.00000	matriz[0][3]=0.00000	matriz[0][4]=1.00000
matriz[1][0]=0.00000	matriz[1][1]=10.00000	matriz[1][2]=-14.00000	matriz[1][3]=0.00000	matriz[1][4]=2.00000
matriz[2][0]=0.00000	matriz[2][1]=-15.00000	matriz[2][2]=21.00000	matriz[2][3]=0.00000	matriz[2][4]=-3.00000


In [59]:
%%bash
./prog_dger.out 3 -1

vector x:
vector[0]=1.00000
vector[1]=2.00000
vector[2]=-3.00000
------------
vector y:
------------
vector resultado:


Parameter 2 to routine DGER   was incorrect


Las dimensiones funcionan igual que en los ejemplos anteriores. En el caso de mandemos más dimensiones el programa llenará esas posiciones con cero. Si mandamos menos posiciones entonces el programa solamente tomará en cuenta las primeras.  
En el ejemplo anterior observamos el caso de que una de las dos dimensiones es menor que cero, a lo que la función no arroja un error.

### Pregunta 4
**Después de haber estudiado y entendido los archivos de definiciones.h y funciones.c y realizado los puntos anteriores y la tarea 5 responde: ¿cómo fue que pudimos llamar a las rutinas de Fortran (que almacena en una forma column-major order los arreglos de dos dimensiones) para operaciones con arreglos 2-dimensionales sin haber instalado CBLAS, si en clase se dijo que almacenar arreglos de dos dimensiones en C es en un row-major order?**


En este programa es posible llamar la función en Fortran por como se ordeno la matriz cuando se mandó llamar. En el archivo `definiciones.h`, en la línea en que se definen las entradas de la matriz se están ordenando las entradas en forma columnar, como se hace en Fortran y no en forma de filas, como es el estándar en C. La línea en el programa se lee de la siguiente manera:  

`#define entrada(arreglo,i,j) ((arreglo)->arr[j*renglones(arreglo)+i]) //almacenamos column major`

En la línea anterior, $i$ es el índice que indica la columna y $j$ es el índice que indica el renglón. Además $N$ es el número de columnas y $M$ es el número de renglones.  
Cuando almacenamos una matriz en un arreglo unidimensional en **formato de renglones** debemos recorrer las entradas como:  
$$entrada(i,j)=i*N+j, \forall i=0,...,N-1 , j=0,...M-1$$   

Cuando almacenamos una matriz en un arreglo unidimensional en **formato de columnas** debemos recorrer las entradas como:  
$$entrada(i,j)=j*M+i, \forall i=0,...,N-1 , j=0,...M-1$$
Esta es la manera en que se recorre el arreglo en la deifción de las entradas de la matriz.  

<img src="column_or_row.png">  

**Imagen 1** Muestra como se recorren los arreglos, del lado izquierdo se recorre de manera columnar (Fortran) y del lado derecho se recorre por renglón (como en C).

### Pregunta 5 
**Investiga* sobre la subrutina de Fortran dgemm (parámetros que recibe y la salida).**

La función DGEMM realiza una multiplicación de matrices y la multiplica por un escalar, de ser necesario transpone una de las matrices y le suma el resultado escalado por una constante. La operación está planteada de la siguiente manera:  
$$C=\alpha * op(A) * op(B) + \beta C_1$$  
donde $op(X)$ puede ser:  
$$op(X)=X \text{    o    } op(X)=X^T$$  
Con $op(A)$ una matriz de $mxk$, $op(B)$ una matriz de $kxn$ $C$ una matriz de $mxn$. $\alpha$ y $\beta$ son escalares.  

Los parámetros que recibe la función son:
* TRANSA (char)- Puede tomar 3 valores: $'N'$ para que no se transponga la matriz $A$. $'T'$ o $'C'$ para que se compute la transpuesta de la matriz $A$. No es sensible a mayúsculas y minúsculas  
* TRANSB (char)- Puede tomar 3 valores: $'N'$ para que no se transponga la matriz $B$. $'T'$ o $'C'$ para que se compute la transpuesta de la matriz $B$. No es sensible a mayúsculas y minúsculas  
* M (int)- Espcifica el número de filas que tiene la matriz $op(A)$. Debe ser al menos cero  
* N (int)- Espcifica el número de columnas que tiene la matriz $op(B)$. Debe ser al menos cero  
* K (int)- Espcifica el número de columnas que tiene la matriz $op(A)$ y el número de filas que tiene la matriz $op(B)$. Debe ser al menos cero  
* ALPHA (double)- Escalar que ajusta la multiplicación de matrices  
* A (double)- Matriz de dimensión $LDAxka$ donde $ka=k$ si $TRANSA='N'$, $m$ si es de otra manera
* LDA (int)- Especifica la primera dimensión de la matriz $A$
* B (double)- Matriz de dimensión $LDBxkb$ donde $kb=n$ si $TRANSB='N'$, $k$ si es de otra manera.
* LDB (int) - Especifica la primera dimensión de la matriz $B$
* BETA (double)- Escalar que multiplica la matriz de salida
* C (double)- Si $\beta=0$ no se necesita especificar. Si $ \beta \neq 0$, $C$ debe de tener dimensiones $mxn$. La salida sobreescribe esta matriz.
* LDC (int)- Especifica la primera dimensión de $C$.

La salida es una matriz $C$ de dimensiones $mxn$, resultado de hacer la operación.

### Pregunta 6
**En la carpeta analisis-numerico-computo-cientifico/C/BLAS/ejemplos/level3/ ejecuta el programa dgemm_mult_mat.c y realiza pruebas con diferentes matrices definidas por ti.**

In [43]:
%%bash
echo 0 1.5 > A.txt
echo 4 -5 >> A.txt
echo -1 2.5 >> A.txt
echo 1 0 0 > B.txt
echo 0 -1 1 >> B.txt

In [6]:
%%file dgemm_mult_mat.c

#include<stdio.h>
#include<stdlib.h>
#include"definiciones.h"
#define A_matriz "A.txt" //de tamaño MxK
#define B_matriz "B.txt" //de tamaño KxN
extern void dgemm_(char *transposeA, char *transposeB,int *m,int *n,int *k,double *alpha,double *A,int *lda,double *B,int *ldb,double *beta,double *C,int *ldc);
int main(int argc, char *argv[]){
	arreglo_2d_T A, B, C;
	int M=atoi(argv[1]);
	int K=atoi(argv[2]);
	int N=atoi(argv[4]);
	double ALPHA, BETA;
    ALPHA = 1.0;
    BETA = 0.0;
	A=malloc(sizeof(*A));
	B=malloc(sizeof(*B));
	C=malloc(sizeof(*C));
	renglones(A)=M;
	columnas(A)=K;
	renglones(B)=K;
	columnas(B)=N;
	renglones(C) = M;
	columnas(C) = N;
	entradas(A)=malloc(renglones(A)*columnas(A)*sizeof(double));
	inicializa_matriz(A,A_matriz);
	entradas(B)=malloc(renglones(B)*columnas(B)*sizeof(double));
	inicializa_matriz(B,B_matriz);
	printf("matriz 1:\n");
	imprime_matriz(A);
	printf("------------\n");
	printf("matriz 2:\n");
	imprime_matriz(B);
	printf("------------\n");
	printf("matriz resultado:\n");
	entradas(C)=calloc(renglones(C)*columnas(C),sizeof(double));
	dgemm_("No transpose", "No transpose", &M, &N, &K, &ALPHA, entradas(A), &M, entradas(B), &K, &BETA, entradas(C), &M);
	imprime_matriz(C);
	free(entradas(A));
	free(A);
	free(entradas(B));
	free(B);
	free(entradas(C));
	free(C);
	return 0;
}

Writing dgemm_mult_mat.c


In [7]:
%%bash 
gcc -Wall dgemm_mult_mat.c funciones.c -o programa.out -lblas

In [13]:
%%bash 
./programa.out 3 2 2 3

matriz 1:
matriz[0][0]=0.00000	matriz[0][1]=1.50000
matriz[1][0]=4.00000	matriz[1][1]=-5.00000
matriz[2][0]=-1.00000	matriz[2][1]=2.50000
------------
matriz 2:
matriz[0][0]=1.00000	matriz[0][1]=0.00000	matriz[0][2]=0.00000
matriz[1][0]=0.00000	matriz[1][1]=-1.00000	matriz[1][2]=1.00000
------------
matriz resultado:
matriz[0][0]=0.00000	matriz[0][1]=-1.50000	matriz[0][2]=1.50000
matriz[1][0]=4.00000	matriz[1][1]=5.00000	matriz[1][2]=-5.00000
matriz[2][0]=-1.00000	matriz[2][1]=-2.50000	matriz[2][2]=2.50000


Hay que notar que estamos mandando el 'No transpose' en la matriz $A$ y en la $B$, lo que indica que para ninguna de las dos matrices se calculará la transpuesta. Además el escalar $\beta$ lo tenemos fijo igual a cero y $\alpha$ igual a $1$. Entonces estamos realizando la operación de multiplicación de matrices sencilla. Las dimensiones $m, n$ y $k$ las estamos mandando como parámetros al programa, estos son los números que se envian como $LDA, LDB \text{ y } LDC$.


In [ ]:
%%bash
echo 3 > A.txt
echo 2  > B.txt

In [17]:
%%bash 
./programa.out 1 1 1 1 

matriz 1:
matriz[0][0]=3.00000
------------
matriz 2:
matriz[0][0]=2.00000
------------
matriz resultado:
matriz[0][0]=6.00000


In [ ]:
Si mandamos un escalar solamente hace la multiplicacion

In [21]:
%%bash
echo 3 2 > A.txt
echo  5 -1>> A.txt
#echo -1 2.5 >> A.txt
echo 2 -1 4  > B.txt
echo 0 -1 -1 >> B.txt
echo 0 1 1 >> B.txt

In [23]:
%%bash 
./programa.out 2 2 32 3

matriz 1:
matriz[0][0]=3.00000	matriz[0][1]=2.00000
matriz[1][0]=5.00000	matriz[1][1]=-1.00000
------------
matriz 2:
matriz[0][0]=2.00000	matriz[0][1]=-1.00000	matriz[0][2]=4.00000
matriz[1][0]=0.00000	matriz[1][1]=-1.00000	matriz[1][2]=-1.00000
------------
matriz resultado:
matriz[0][0]=6.00000	matriz[0][1]=-5.00000	matriz[0][2]=10.00000
matriz[1][0]=10.00000	matriz[1][1]=-4.00000	matriz[1][2]=21.00000


Aunque mandemos dimensiones que no concuerdan se realiza la operación. Lo que esta pasando es que el tercer argumento posicional que enviamos no lo usamos para nada en el programa. El segundo argumento posicional determina la $k$ que corresponde a la segunda dimensión de la matriz $A$ y a la primera de la matriz $B$. Entonces el programa detectó $A$ de $2x2$ y $B$ de $2x3$, entonces el resultado es de $2x3$.

### Pregunta 7
**En la carpeta del punto anterior encuentras la sección Multiplicación matriz-matriz con trick. Ejecuta el programa de esta sección con diferentes matrices definidas por ti y resuelve la pregunta ¿por qué funciona este trick?.**


In [37]:
%%file definiciones_trick.h

typedef struct{
int m, n;
#define renglones(arreglo) ((arreglo)->m)
#define columnas(arreglo) ((arreglo)->n)
double *arr;
#define entradas(arreglo) ((arreglo)->arr)
#define entrada(arreglo,i,j) ((arreglo)->arr[i*columnas(arreglo)+j]) //almacenamos row major
}arreglo_2d;
typedef arreglo_2d *arreglo_2d_T;
typedef struct{
int n;
#define renglones_vector(arreglo) ((arreglo)->n)
double *arr;
int *arr_int;
#define entradas_vector(arreglo) ((arreglo)->arr)
#define entrada_vector(arreglo,i) ((arreglo)->arr[i])
#define entradas_vector_entero(arreglo) ((arreglo)->arr_int)
#define entrada_vector_entero(arreglo,i) ((arreglo)->arr_int[i])
}arreglo_1d;
typedef arreglo_1d *arreglo_1d_T;
void imprime_vector(arreglo_1d_T);
void imprime_matriz(arreglo_2d_T);
void inicializa_matriz(arreglo_2d_T, char *);
void inicializa_vector(arreglo_1d_T, char *);
void imprime_vector_entero(arreglo_1d_T);

Overwriting definiciones_trick.h


In [36]:
%%file dgemm_mult_mat_trick.c

#include<stdio.h>
#include<stdlib.h>
#include"definiciones_trick.h"
#define A_matriz "A.txt" //de tamaño MxK
#define B_matriz "B.txt" //de tamaño KxN
extern void dgemm_(char *transposeA, char *transposeB,int *m,int *n,int *k,double *alpha,double *A,int *lda,double *B,int *ldb,double *beta,double *C,int *ldc);
int main(int argc, char *argv[]){
	arreglo_2d_T A, B, C;
	int M=atoi(argv[1]);
	int K=atoi(argv[2]);
	int N=atoi(argv[4]);
	double ALPHA, BETA;
    ALPHA = 1.0;
    BETA = 0.0;
	A=malloc(sizeof(*A));
	B=malloc(sizeof(*B));
	C=malloc(sizeof(*C));
	renglones(A)=M;
	columnas(A)=K;
	renglones(B)=K;
	columnas(B)=N;
	renglones(C) = M;
	columnas(C) = N;
	entradas(A)=malloc(renglones(A)*columnas(A)*sizeof(double));
	inicializa_matriz(A,A_matriz);
	entradas(B)=malloc(renglones(B)*columnas(B)*sizeof(double));
	inicializa_matriz(B,B_matriz);
	printf("matriz 1:\n");
	imprime_matriz(A);
	printf("------------\n");
	printf("matriz 2:\n");
	imprime_matriz(B);
	printf("matriz 3:\n");
	entradas(C)=calloc(renglones(C)*columnas(C),sizeof(double));
	dgemm_("No transpose", "No transpose", &N, &M, &K, &ALPHA, entradas(B), &N, entradas(A), &K, &BETA, entradas(C), &N);
	imprime_matriz(C);
	free(entradas(A));
	free(A);
	free(entradas(B));
	free(B);
	free(entradas(C));
	free(C);
	return 0;
}

Overwriting dgemm_mult_mat_trick.c


In [39]:
%%file funciones_trick.c
    
#include<stdio.h>
#include<stdlib.h>
#include"definiciones_trick.h"
static int i,j;
void inicializa_matriz(arreglo_2d_T p, char *s){
	int m = renglones(p);
	int n = columnas(p);
	FILE * pFile;
  	pFile = fopen (s,"r");
  	for(i=0;i<m;i++)
		for(j=0;j<n;j++)
			fscanf(pFile,"%lf", &entrada(p,i,j));
	fclose(pFile);
}
void inicializa_vector(arreglo_1d_T p, char *s){
	int m = renglones_vector(p);
	FILE * pFile;
  	pFile = fopen (s,"r");
  	for(i=0;i<m;i++)
			fscanf(pFile,"%lf", &entrada_vector(p,i));
	fclose(pFile);
}
void imprime_matriz(arreglo_2d_T p){
	int m = renglones(p);
	int n = columnas(p);
		for(i=0;i<m;i++){
			for(j=0;j<n;j++){
				if(j<n-1)
				printf("matriz[%d][%d]=%.5f\t",i,j,entrada(p,i,j));
				else
				printf("matriz[%d][%d]=%.5f\n",i,j,entrada(p,i,j));
			}
		}
}
void imprime_vector_entero(arreglo_1d_T p){
	int m = renglones_vector(p);
		for(i=0;i<m;i++)
				printf("vector[%d]=%d\n",i,entrada_vector_entero(p,i));
}
void imprime_vector(arreglo_1d_T p){
	int m = renglones_vector(p);
			for(i=0;i<m;i++)
				printf("vector[%d]=%.5f\n",i,entrada_vector(p,i));
}

Writing funciones_trick.c


In [41]:
%%bash                           
gcc -Wall dgemm_mult_mat_trick.c funciones_trick.c -o programa_trick.out -lblas

In [45]:
%%bash 
./programa_trick.out 3 2 2 3

matriz 1:
matriz[0][0]=0.00000	matriz[0][1]=1.50000
matriz[1][0]=4.00000	matriz[1][1]=-5.00000
matriz[2][0]=-1.00000	matriz[2][1]=2.50000
------------
matriz 2:
matriz[0][0]=1.00000	matriz[0][1]=0.00000	matriz[0][2]=0.00000
matriz[1][0]=0.00000	matriz[1][1]=-1.00000	matriz[1][2]=1.00000
matriz 3:
matriz[0][0]=0.00000	matriz[0][1]=-1.50000	matriz[0][2]=1.50000
matriz[1][0]=4.00000	matriz[1][1]=5.00000	matriz[1][2]=-5.00000
matriz[2][0]=-1.00000	matriz[2][1]=-2.50000	matriz[2][2]=2.50000


El trick funciona por que como estamos definiendo y guardando las matrices es equivalente a estar calculando la matriz transpuesta. Independientemente de como capturemos las entradas, Fortran las seguirá tomando como si fuera columnar, entonces al en formato de filas (llenando elemento a elemento) es como haberlo capturado normal y haber calculdo la transpuesta.  
Luego alimentamos a la función cambiando el orden de las matrices a multiplicar. Como sabemos, para tres matrices $A$, $B$ y $C$:  
$$C=AB \implies{} C^T=(AB)^T=B^T A^T$$  
Entonces mandamos a la función en Fortran las matrices $A$ y $B$ guardadas en formato de filas, en Fortran es similar a haber calculado las transpuestas y al multiplicar se calcula $C^T$. Al imprimir, el método funciona con el método columnar, entonces al recorrer por columnas la matriz que está guardada por filas vuelve a ser equivalente a calcular la transpuesta y el programa imprime $C$ sin transponer.   
También es importante notar que a la función enviamos las dimensiones de las matrices como si estuvieran transpuestas. 
* Envaimos $N$ como la primera dimensión de $B$, cuando en realidad es $k$  
* Envaimos $k$ como la primera dimensión de $A$, cuando en realidad es $M$  
* Envaimos $N$ como la primera dimensión de $C$, cuando en realidad es $M$  
* Envaimos $M$ al parámetro $N$
* Envaimos $N$ al parámetro $M$


### Pregunta 8
**Haz un programa que utilice la subrutina dsymm de Fortran.**



La función **DSYMM** hace una multiplicación de una matriz simétrica por otra matriz. Solamente usa una de las matrices triangulares asociadas a la simétrica para caluclarla. Se indica como parámtero que matriz triangular usará el programa.

In [48]:
%%file dsymm.c

#include<stdio.h>
#include<stdlib.h>
#include"definiciones.h"
#define A_matriz "A.txt" //de tamaño MxK
#define B_matriz "B.txt" //de tamaño KxN
extern void dsymm_(char *side, char *uplo, int *m,int *n, double *alpha,double *A,int *lda,double *B,int *ldb,double *beta,double *C,int *ldc);
int main(int argc, char *argv[]){
	arreglo_2d_T A, B, C;
	int M=atoi(argv[1]);
	int N=atoi(argv[2]);
	double ALPHA, BETA;
    ALPHA = 1.0;
    BETA = 0.0;
	A=malloc(sizeof(*A));
	B=malloc(sizeof(*B));
	C=malloc(sizeof(*C));
	renglones(A)=M;
	columnas(A)=M;
	renglones(B)=M;
	columnas(B)=N;
	renglones(C) = M;
	columnas(C) = N;
	entradas(A)=malloc(renglones(A)*columnas(A)*sizeof(double));
	inicializa_matriz(A,A_matriz);
	entradas(B)=malloc(renglones(B)*columnas(B)*sizeof(double));
	inicializa_matriz(B,B_matriz);
	printf("matriz 1:\n");
	imprime_matriz(A);
	printf("------------\n");
	printf("matriz 2:\n");
	imprime_matriz(B);
	printf("------------\n");
	printf("matriz resultado:\n");
	entradas(C)=calloc(renglones(C)*columnas(C),sizeof(double));
	dsymm_("L", "U", &M, &N, &ALPHA, entradas(A), &M, entradas(B), &M, &BETA, entradas(C), &M);
	imprime_matriz(C);
	free(entradas(A));
	free(A);
	free(entradas(B));
	free(B);
	free(entradas(C));
	free(C);
	return 0;
}

Overwriting dsymm.c


In [49]:
%%bash 
gcc -Wall dsymm.c funciones.c -o prog_dsymm.out -lblas

In [52]:
%%bash
#matriz A
echo -8 -1 3 >A.txt
echo -1 7 4 >>A.txt
echo -3 4 9 >>A.txt

# matriz B
echo  1 4 > B.txt
echo -1 3 >>B.txt
echo -3 2 >>B.txt


In [53]:
%%bash 
./prog_dsymm.out 3 2 

matriz 1:
matriz[0][0]=-8.00000	matriz[0][1]=-1.00000	matriz[0][2]=3.00000
matriz[1][0]=-1.00000	matriz[1][1]=7.00000	matriz[1][2]=4.00000
matriz[2][0]=-3.00000	matriz[2][1]=4.00000	matriz[2][2]=9.00000
------------
matriz 2:
matriz[0][0]=1.00000	matriz[0][1]=4.00000
matriz[1][0]=-1.00000	matriz[1][1]=3.00000
matriz[2][0]=-3.00000	matriz[2][1]=2.00000
------------
matriz resultado:
matriz[0][0]=-16.00000	matriz[0][1]=-29.00000
matriz[1][0]=-20.00000	matriz[1][1]=25.00000
matriz[2][0]=-28.00000	matriz[2][1]=42.00000


**Imagen extraída de:  
https://software.intel.com/en-us/mkl-macos-developer-guide-calling-lapack-blas-and-cblas-routines-from-c-c-language-environments